# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7efef880f550>
├── 'a' --> tensor([[-1.2227,  1.0143, -0.6879],
│                   [ 1.0516,  1.5267, -0.8344]])
└── 'x' --> <FastTreeValue 0x7efef880f190>
    └── 'c' --> tensor([[ 0.2029,  1.3736, -0.1720,  0.6017],
                        [ 0.6972,  2.6870,  0.0113,  1.2235],
                        [ 0.0092, -0.4886,  1.4256, -0.0810]])

In [4]:
t.a

tensor([[-1.2227,  1.0143, -0.6879],
        [ 1.0516,  1.5267, -0.8344]])

In [5]:
%timeit t.a

64.5 ns ± 0.495 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7efef880f550>
├── 'a' --> tensor([[ 0.7179,  0.9966,  0.4454],
│                   [-1.1645,  0.1632,  1.6681]])
└── 'x' --> <FastTreeValue 0x7efef880f190>
    └── 'c' --> tensor([[ 0.2029,  1.3736, -0.1720,  0.6017],
                        [ 0.6972,  2.6870,  0.0113,  1.2235],
                        [ 0.0092, -0.4886,  1.4256, -0.0810]])

In [7]:
%timeit t.a = new_value

64.9 ns ± 1.64 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.2227,  1.0143, -0.6879],
               [ 1.0516,  1.5267, -0.8344]]),
    x: Batch(
           c: tensor([[ 0.2029,  1.3736, -0.1720,  0.6017],
                      [ 0.6972,  2.6870,  0.0113,  1.2235],
                      [ 0.0092, -0.4886,  1.4256, -0.0810]]),
       ),
)

In [10]:
b.a

tensor([[-1.2227,  1.0143, -0.6879],
        [ 1.0516,  1.5267, -0.8344]])

In [11]:
%timeit b.a

64.6 ns ± 0.0605 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3899,  1.0452, -0.5164],
               [-1.2585, -1.1084, -0.5726]]),
    x: Batch(
           c: tensor([[ 0.2029,  1.3736, -0.1720,  0.6017],
                      [ 0.6972,  2.6870,  0.0113,  1.2235],
                      [ 0.0092, -0.4886,  1.4256, -0.0810]]),
       ),
)

In [13]:
%timeit b.a = new_value

483 ns ± 2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

838 ns ± 0.379 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 17 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 511 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 788 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7efef87951f0>
├── 'a' --> tensor([[[-1.2227,  1.0143, -0.6879],
│                    [ 1.0516,  1.5267, -0.8344]],
│           
│                   [[-1.2227,  1.0143, -0.6879],
│                    [ 1.0516,  1.5267, -0.8344]],
│           
│                   [[-1.2227,  1.0143, -0.6879],
│                    [ 1.0516,  1.5267, -0.8344]],
│           
│                   [[-1.2227,  1.0143, -0.6879],
│                    [ 1.0516,  1.5267, -0.8344]],
│           
│                   [[-1.2227,  1.0143, -0.6879],
│                    [ 1.0516,  1.5267, -0.8344]],
│           
│                   [[-1.2227,  1.0143, -0.6879],
│                    [ 1.0516,  1.5267, -0.8344]],
│           
│                   [[-1.2227,  1.0143, -0.6879],
│                    [ 1.0516,  1.5267, -0.8344]],
│           
│                   [[-1.2227,  1.0143, -0.6879],
│                    [ 1.0516,  1.5267, -0.8344]]])
└── 'x' --> <FastTreeValue 0x7efef87957f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 44.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7efef87952e0>
├── 'a' --> tensor([[-1.2227,  1.0143, -0.6879],
│                   [ 1.0516,  1.5267, -0.8344],
│                   [-1.2227,  1.0143, -0.6879],
│                   [ 1.0516,  1.5267, -0.8344],
│                   [-1.2227,  1.0143, -0.6879],
│                   [ 1.0516,  1.5267, -0.8344],
│                   [-1.2227,  1.0143, -0.6879],
│                   [ 1.0516,  1.5267, -0.8344],
│                   [-1.2227,  1.0143, -0.6879],
│                   [ 1.0516,  1.5267, -0.8344],
│                   [-1.2227,  1.0143, -0.6879],
│                   [ 1.0516,  1.5267, -0.8344],
│                   [-1.2227,  1.0143, -0.6879],
│                   [ 1.0516,  1.5267, -0.8344],
│                   [-1.2227,  1.0143, -0.6879],
│                   [ 1.0516,  1.5267, -0.8344]])
└── 'x' --> <FastTreeValue 0x7efe40a14a30>
    └── 'c' --> tensor([[ 0.2029,  1.3736, -0.1720,  0.6017],
                        [ 0.6972,  2.6870,  0.0113,  1.2235],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 85.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.9 µs ± 57.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2029,  1.3736, -0.1720,  0.6017],
                       [ 0.6972,  2.6870,  0.0113,  1.2235],
                       [ 0.0092, -0.4886,  1.4256, -0.0810]],
              
                      [[ 0.2029,  1.3736, -0.1720,  0.6017],
                       [ 0.6972,  2.6870,  0.0113,  1.2235],
                       [ 0.0092, -0.4886,  1.4256, -0.0810]],
              
                      [[ 0.2029,  1.3736, -0.1720,  0.6017],
                       [ 0.6972,  2.6870,  0.0113,  1.2235],
                       [ 0.0092, -0.4886,  1.4256, -0.0810]],
              
                      [[ 0.2029,  1.3736, -0.1720,  0.6017],
                       [ 0.6972,  2.6870,  0.0113,  1.2235],
                       [ 0.0092, -0.4886,  1.4256, -0.0810]],
              
                      [[ 0.2029,  1.3736, -0.1720,  0.6017],
                       [ 0.6972,  2.6870,  0.0113,  1.2235],
                       [ 0.0092, -0.4886,  1.4256, -0.0810]],

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 83.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2029,  1.3736, -0.1720,  0.6017],
                      [ 0.6972,  2.6870,  0.0113,  1.2235],
                      [ 0.0092, -0.4886,  1.4256, -0.0810],
                      [ 0.2029,  1.3736, -0.1720,  0.6017],
                      [ 0.6972,  2.6870,  0.0113,  1.2235],
                      [ 0.0092, -0.4886,  1.4256, -0.0810],
                      [ 0.2029,  1.3736, -0.1720,  0.6017],
                      [ 0.6972,  2.6870,  0.0113,  1.2235],
                      [ 0.0092, -0.4886,  1.4256, -0.0810],
                      [ 0.2029,  1.3736, -0.1720,  0.6017],
                      [ 0.6972,  2.6870,  0.0113,  1.2235],
                      [ 0.0092, -0.4886,  1.4256, -0.0810],
                      [ 0.2029,  1.3736, -0.1720,  0.6017],
                      [ 0.6972,  2.6870,  0.0113,  1.2235],
                      [ 0.0092, -0.4886,  1.4256, -0.0810],
                      [ 0.2029,  1.3736, -0.1720,  0.6017],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 2.34 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
